<a href="https://colab.research.google.com/github/ua-datalab/Geospatial_Workshops/notebooks/blob/main/Planet_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This Notebook demonstrates how to use python to tap into Planet's API to search and download imagery data.

Written by Jeffrey Gillan, 2024 (with help from ChatGPT4.0)

## Setup

In [ ]:
# Determine the python version we are working in
import sys
print(sys.version)

In [ ]:
#Colab comes with many python libraries pre-installed
#You can view the list of installed libraries
!pip freeze

In [3]:
#Import libraries to use into our environment

import os
import json
import requests
import time
from requests.auth import HTTPBasicAuth

In [4]:
# Helper function to print formatted Json using the json module. This makes reading json format much easier.
def p(data):
    print(json.dumps(data, indent=2))

In [7]:
#Set your API Key as a variable. Logon to Planet https://www.planet.com/login and go to 'My Settings' to get key.
#An API key is similar to a password. It tells Planet that your are a registered user.

API_KEY = ''

In [8]:
# Defining API URL Endpoints. 'Endpoints' are the web addresses where the API is located.
# Different endpoints do slight different things. The 3 endpoints listed here (stats, quick-search, orders) each provide different information to the user.

# The Data API is used to search for and get stats on imagery items. You can only download 1 image at a time here
stats_url = "https://api.planet.com/data/v1/stats"
quick_search_url = "https://api.planet.com/data/v1/quick-search"

# The Orders API is used for bulk download of imagery
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

## Search and Download Imagery using the Planet Data API

In [5]:
### Create an Area of Interest (AOI) that you want to get satellite imagery from

#Tucson Arizona bounding box (https://geojson.io/)
geojson_geometry = {
        "coordinates": [
          [
            [
              -110.92350539668898,
              32.518041630963125
            ],
            [
              -110.92350539668898,
              32.336129360270974
            ],
            [
              -110.6292017202573,
              32.336129360270974
            ],
            [
              -110.6292017202573,
              32.518041630963125
            ],
            [
              -110.92350539668898,
              32.518041630963125
            ]
          ]
        ],
        "type": "Polygon"
      }

In [6]:
###Define Filtering Parameters for Searching for Planet Imagery (https://developers.planet.com/docs/apis/data/searches-filtering/)

# get images that overlap with our AOI
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",   #Type of filter = Data Range
  "field_name": "acquired",
  "config": {
    "gte": "2020-06-18T00:00:00.000Z", # gte = greater than or equals
    "lte": "2020-06-19T00:00:00.000Z"  # lte = less than or equals
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5  # Less than or equal to 50% cloud cover
  }
}

# combine our geo, date, cloud filters into a single filter
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [ ]:
## Send Filtered Search Request to Planet API to get the Number of Imagery Assets

#Create API Request object
item_type = "PSScene" # we are looking for PlanetScope Imagery (https://www.planet.com/products/planet-imagery/)
stat_request = {
    "item_types" : [item_type],
    "interval" : "day",
    "filter" : combined_filter
}

# Send the POST request to the API stats endpoint
stat_response = \
  requests.post(
      stats_url,       #Using the 'stats' endpoint address
      auth=HTTPBasicAuth(API_KEY, ''),
      json=stat_request)

geojson = stat_response.json()

p(geojson)

In [ ]:
### Send Filtered Search Request to Planet API to Get Imagery Names

# Create API request object
item_type = "PSScene" # we are looking for PlanetScope Imagery (https://www.planet.com/products/planet-imagery/)
search_request = {
  "item_types": [item_type],
  "filter": combined_filter
}


# Send POST request to Planet API
search_result = \
  requests.post(
    quick_search_url,   # Send to the quick-search endpoint address
    auth=HTTPBasicAuth(API_KEY, ''),
    json=search_request)

geojson = search_result.json()

# let's look at the first result
p(list(geojson.items())[1][1][0])

In [ ]:
# extract image IDs only
image_ids = [feature['id'] for feature in geojson['features']]
p(image_ids)

In [ ]:
# Let's Pick an interesting image

id0 = '20200618_175631_92_106d'

id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)

# Use GET request to see the specific assets for the item ID
result = \
  requests.get(
    id0_url,
    auth=HTTPBasicAuth(API_KEY, '')
  )

# List of asset types available for this particular satellite image
print(result.json().keys())

In [ ]:
### If Imagery Assets have not been touched in 30 days, they are in cold storage and need to be activated

# Check the status of the Asset
print(result.json()['ortho_analytic_4b']['status'])

In [16]:
### Send a request to have the image activated

# Parse out useful links
links = result.json()[u"ortho_analytic_4b"]["_links"]
self_link = links["_self"]
activation_link = links["activate"]

# Request activation of the 'ortho_analytic_4b' asset:
activate_result = \
  requests.get(
    activation_link,
    auth=HTTPBasicAuth(API_KEY, '')
  )

In [ ]:
##Check the status of the asset now

activation_status_result = \
  requests.get(
    self_link,
    auth=HTTPBasicAuth(API_KEY, '')
  )

print(activation_status_result.json()["status"])

In [ ]:
# Get the download link for the active asset
download_link = activation_status_result.json()["location"]
print(download_link)

In [ ]:
##Download the image into Colab disk

# Send a GET request to the download URL
response = requests.get(download_link)

# Check if the request was successful
if response.status_code == 200:
    # Write the content of the response to a file
    # Replace 'image_name_extension.' with the appropriate name and extension for your image
    with open('bighorn_fire.tif', 'wb') as file:
        file.write(response.content)
    print("Image downloaded successfully!")
else:
    print("Failed to download the image. Status code:", response.status_code)

# Now you can load and process your image using the appropriate libraries and tools

## Visualize Satellite Imagery

In [ ]:
## Install libraries for visualization and analysis

!pip install rasterio
!pip install GDAL

## bring the libraries into the environment
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

In [22]:
# Set a variable containing the path of the imagery on your Colab disk
image_path = '/content/bighorn_fire.tif'  # Replace with your file path


In [ ]:
### Visualize the Satellite Imagery
## Planetscope band order: band 1 = blue; band 2 = green; band 3 = red; band 4 = near-infrared

with rasterio.open(image_path) as src:
    image = src.read([4, 3, 2])  # Display as false-color composite (show nir as red; show red as green; show green as blue)

# Normalize the bands
image = image.astype('float32')
for i in range(image.shape[0]):
    image[i] /= image[i].max()

plt.figure(figsize=(10, 10))

image = image.transpose(1, 2, 0)  # Transpose dimensions
plt.imshow(image)
plt.axis('off')
plt.show()

## Download multiple imagery assets using Planet Orders API

In [24]:
#Define the item IDs and asset types I want to download

multi_products = [
    {
        "item_ids":[
            "20200618_175633_99_106d",
            "20200618_175631_92_106d",
            "20200618_175629_85_106d"
         ],
         "item_type":"PSScene",
         "product_bundle":"analytic_udm2"     #asset type
    }
]

In [25]:
##Create API Request object

order_request = {
    "name": "mt_lemmon",
    "products": multi_products,
    "deliver": {"single_archive": True, "archive_type": "zip"}
}

In [ ]:
order_request

In [ ]:
# Send the POST request to the Orders API endpoints
order_response = \
  requests.post(
      orders_url,
      auth=HTTPBasicAuth(API_KEY, ''),
      json=order_request)

geojson = order_response.json()

p(geojson)

In [ ]:
##Check if your order is ready for download

# The url is the Orders API address + the order 'ID' provided in the order response
order_status_url = "https://api.planet.com/compute/ops/orders/v2/cf49ada4-b4d4-4efe-953e-02d541d6af67"  # Replace with your order status URL

while True:
    response = requests.get(order_status_url, auth=HTTPBasicAuth(API_KEY, ''))
    response.raise_for_status()
    order_status = response.json()

    if order_status['state'] in ['success', 'failed']:
        break
    else:
        print("Order is still processing...")
        time.sleep(60)  # Wait for 60 seconds before checking again

if order_status['state'] == 'success':
    print("Order is ready!")
elif order_status['state'] == 'failed':
    print("Order failed:", order_status.get('error', 'No error message provided'))


In [ ]:
order_status

In [ ]:
##Download the order to Colab

def download_file(url, local_filename):
    with requests.get(url, stream=True, auth=HTTPBasicAuth(API_KEY, '')) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)


if order_status['state'] == 'success':
    for file_info in order_status['_links']['results']:
        file_url = file_info['location']
        full_name = file_info['name']  # The full 'name' field with slashes
        file_name = full_name.split('/')[-1]  # Extracting the last part after the last slash

        # Construct the local file path
        local_filename = os.path.join("/content", file_name)

        download_file(file_url, local_filename)
        print(f"Downloaded {local_filename}")
